In [ ]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/ib6j1thnwarewyhir00ha


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
import swanlab
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np

swanlab.init(project="plant-disease-detection", run="se_resnet50_new2")

# 定义SE模块
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# 包装 torchvision 自带 Bottleneck 加 SE
class SEBottleneck(nn.Module):
    def __init__(self, bottleneck):
        super(SEBottleneck, self).__init__()
        self.bottleneck = bottleneck
        self.se = SEBlock(bottleneck.conv3.out_channels)

    def forward(self, x):
        out = self.bottleneck(x)
        out = self.se(out)
        return out

# 构建 SE-ResNet50
def build_se_resnet50(num_classes):
    model = models.resnet50(pretrained=False)
    for layer_name in ['layer1', 'layer2', 'layer3', 'layer4']:
        old_layer = getattr(model, layer_name)
        new_blocks = []
        for block in old_layer:
            new_blocks.append(SEBottleneck(block))
        setattr(model, layer_name, nn.Sequential(*new_blocks))
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Albumentations 增强定义
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0)),
    ToTensorV2(),
])


val_transform = A.Compose([
    A.Resize(224, 224),
    ToTensorV2(),
])

# 自定义 Dataset 兼容 Albumentations
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        from torchvision.datasets import ImageFolder
        self.dataset = ImageFolder(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

train_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/train', transform=train_transform)
val_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/val', transform=val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=36,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_se_resnet50(num_classes=38).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

scaler = GradScaler()

def train(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss_total, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    out = model(x)
                    loss = criterion(out, y)
                val_loss_total += loss.item()
                val_correct += (out.argmax(1) == y).sum().item()
                val_total += y.size(0)
        val_loss = val_loss_total / len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
        swanlab.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

train(model, train_loader, val_loader, epochs=20)
torch.save(model.state_dict(), "se_resnet50_new.pth")


swanlab: Tracking run with swanlab version 0.6.0                                   
swanlab: Run data will be saved locally in /swanlog/run-20250604_042207-80449d08
swanlab: 👋 Hi SZY_230507, welcome to swanlab!
swanlab: Syncing run horse-8 to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/59sp0pw6tkuykplu6tgin


ValueError: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.8, 1.0), 'ra...': 1.0, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/59sp0pw6tkuykplu6tgin


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
import swanlab
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np

swanlab.init(project="plant-disease-detection", run="se_resnet50_new2")

# 定义SE模块
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# 包装 torchvision 自带 Bottleneck 加 SE
class SEBottleneck(nn.Module):
    def __init__(self, bottleneck):
        super(SEBottleneck, self).__init__()
        self.bottleneck = bottleneck
        self.se = SEBlock(bottleneck.conv3.out_channels)

    def forward(self, x):
        out = self.bottleneck(x)
        out = self.se(out)
        return out

# 构建 SE-ResNet50
def build_se_resnet50(num_classes):
    model = models.resnet50(pretrained=False)
    for layer_name in ['layer1', 'layer2', 'layer3', 'layer4']:
        old_layer = getattr(model, layer_name)
        new_blocks = []
        for block in old_layer:
            new_blocks.append(SEBottleneck(block))
        setattr(model, layer_name, nn.Sequential(*new_blocks))
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Albumentations 增强定义
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    ToTensorV2(),
])



val_transform = A.Compose([
    A.Resize(224, 224),
    ToTensorV2(),
])

# 自定义 Dataset 兼容 Albumentations
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        from torchvision.datasets import ImageFolder
        self.dataset = ImageFolder(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

train_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/train', transform=train_transform)
val_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/val', transform=val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=36,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_se_resnet50(num_classes=38).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

scaler = GradScaler()

def train(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss_total, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    out = model(x)
                    loss = criterion(out, y)
                val_loss_total += loss.item()
                val_correct += (out.argmax(1) == y).sum().item()
                val_total += y.size(0)
        val_loss = val_loss_total / len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
        swanlab.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

train(model, train_loader, val_loader, epochs=20)
torch.save(model.state_dict(), "se_resnet50_new.pth")


swanlab: Tracking run with swanlab version 0.6.0                                   
swanlab: Run data will be saved locally in /swanlog/run-20250604_042324-12595fe6
swanlab: 👋 Hi SZY_230507, welcome to swanlab!
swanlab: Syncing run goat-9 to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/m85xo8qr3nxkrny71316c


ValueError: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.8, 1.0), 'ra...': 1.0, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/m85xo8qr3nxkrny71316c


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
import swanlab
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np

swanlab.init(project="plant-disease-detection", run="se_resnet50_new2")

# 定义SE模块
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# 包装 torchvision 自带 Bottleneck 加 SE
class SEBottleneck(nn.Module):
    def __init__(self, bottleneck):
        super(SEBottleneck, self).__init__()
        self.bottleneck = bottleneck
        self.se = SEBlock(bottleneck.conv3.out_channels)

    def forward(self, x):
        out = self.bottleneck(x)
        out = self.se(out)
        return out

# 构建 SE-ResNet50
def build_se_resnet50(num_classes):
    model = models.resnet50(pretrained=False)
    for layer_name in ['layer1', 'layer2', 'layer3', 'layer4']:
        old_layer = getattr(model, layer_name)
        new_blocks = []
        for block in old_layer:
            new_blocks.append(SEBottleneck(block))
        setattr(model, layer_name, nn.Sequential(*new_blocks))
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Albumentations 增强定义
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    ToTensorV2(),
])



val_transform = A.Compose([
    A.Resize(224, 224),
    ToTensorV2(),
])

# 自定义 Dataset 兼容 Albumentations
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        from torchvision.datasets import ImageFolder
        self.dataset = ImageFolder(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

train_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/train', transform=train_transform)
val_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/val', transform=val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=36,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_se_resnet50(num_classes=38).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

scaler = GradScaler()

def train(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss_total, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    out = model(x)
                    loss = criterion(out, y)
                val_loss_total += loss.item()
                val_correct += (out.argmax(1) == y).sum().item()
                val_total += y.size(0)
        val_loss = val_loss_total / len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
        swanlab.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

train(model, train_loader, val_loader, epochs=20)
torch.save(model.state_dict(), "se_resnet50_new.pth")


swanlab: Tracking run with swanlab version 0.6.0                                   
swanlab: Run data will be saved locally in /swanlog/run-20250604_042540-eab6bfb6
swanlab: 👋 Hi SZY_230507, welcome to swanlab!
swanlab: Syncing run snake-7 to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/hjn38wo8qzmoweeijngpo


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_33203/452958575.py:140: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1 Train:   0%|          | 0/3385 [00:00<?, ?it/s]/tmp/ipykernel_33203/452958575.py:149: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 Train:   0%|         

RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.HalfTensor) should be the same

In [ ]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/hjn38wo8qzmoweeijngpo


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
import swanlab
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np

swanlab.init(project="plant-disease-detection", run="se_resnet50_new2")

# 定义SE模块
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# 包装 torchvision 自带 Bottleneck 加 SE
class SEBottleneck(nn.Module):
    def __init__(self, bottleneck):
        super(SEBottleneck, self).__init__()
        self.bottleneck = bottleneck
        self.se = SEBlock(bottleneck.conv3.out_channels)

    def forward(self, x):
        out = self.bottleneck(x)
        out = self.se(out)
        return out

# 构建 SE-ResNet50
def build_se_resnet50(num_classes):
    model = models.resnet50(pretrained=False)
    for layer_name in ['layer1', 'layer2', 'layer3', 'layer4']:
        old_layer = getattr(model, layer_name)
        new_blocks = []
        for block in old_layer:
            new_blocks.append(SEBottleneck(block))
        setattr(model, layer_name, nn.Sequential(*new_blocks))
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Albumentations 增强定义
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet mean/std
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])


# 自定义 Dataset 兼容 Albumentations
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        from torchvision.datasets import ImageFolder
        self.dataset = ImageFolder(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

train_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/train', transform=train_transform)
val_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/val', transform=val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=36,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_se_resnet50(num_classes=38).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

scaler = GradScaler()

def train(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss_total, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    out = model(x)
                    loss = criterion(out, y)
                val_loss_total += loss.item()
                val_correct += (out.argmax(1) == y).sum().item()
                val_total += y.size(0)
        val_loss = val_loss_total / len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
        swanlab.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

train(model, train_loader, val_loader, epochs=20)
torch.save(model.state_dict(), "se_resnet50_new.pth")


swanlab: Tracking run with swanlab version 0.6.0                                   
swanlab: Run data will be saved locally in /swanlog/run-20250604_042850-201ef10f
swanlab: 👋 Hi SZY_230507, welcome to swanlab!
swanlab: Syncing run horse-8 to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/iwujj4o8s4xkbjms0dnyp


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_33203/1317736668.py:141: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1 Train:   0%|          | 0/3385 [00:00<?, ?it/s]/tmp/ipykernel_33203/1317736668.py:150: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 Train:   3%|▎      

KeyboardInterrupt: 

In [ ]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/iwujj4o8s4xkbjms0dnyp


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
import swanlab
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np

swanlab.init(project="plant-disease-detection", run="se_resnet50_new2")

# 定义SE模块
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# 包装 torchvision 自带 Bottleneck 加 SE
class SEBottleneck(nn.Module):
    def __init__(self, bottleneck):
        super(SEBottleneck, self).__init__()
        self.bottleneck = bottleneck
        self.se = SEBlock(bottleneck.conv3.out_channels)

    def forward(self, x):
        out = self.bottleneck(x)
        out = self.se(out)
        return out

# 构建 SE-ResNet50
def build_se_resnet50(num_classes):
    model = models.resnet50(pretrained=False)
    for layer_name in ['layer1', 'layer2', 'layer3', 'layer4']:
        old_layer = getattr(model, layer_name)
        new_blocks = []
        for block in old_layer:
            new_blocks.append(SEBottleneck(block))
        setattr(model, layer_name, nn.Sequential(*new_blocks))
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Albumentations 增强定义
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet mean/std
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])


# 自定义 Dataset 兼容 Albumentations
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        from torchvision.datasets import ImageFolder
        self.dataset = ImageFolder(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

train_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/train', transform=train_transform)
val_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/val', transform=val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=36,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_se_resnet50(num_classes=38).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

scaler = GradScaler()

def train(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss_total, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    out = model(x)
                    loss = criterion(out, y)
                val_loss_total += loss.item()
                val_correct += (out.argmax(1) == y).sum().item()
                val_total += y.size(0)
        val_loss = val_loss_total / len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
        swanlab.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

train(model, train_loader, val_loader, epochs=20)
torch.save(model.state_dict(), "se_resnet50_new.pth")


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


swanlab: Tracking run with swanlab version 0.6.0                                   
swanlab: Run data will be saved locally in /swanlog/run-20250604_060044-bc730261
swanlab: 👋 Hi SZY_230507, welcome to swanlab!
swanlab: Syncing run goat-9 to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/lmmov5q7hwb2hck3b48rp


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1277/1317736668.py:141: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1 Train:   0%|          | 0/3385 [00:00<?, ?it/s]/tmp/ipykernel_1277/1317736668.py:150: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 Train: 100%|█████████

Epoch 1: Train Loss=1.6019, Train Acc=0.5276, Val Loss=0.7185, Val Acc=0.7697


Epoch 2 Train: 100%|██████████| 3385/3385 [08:33<00:00,  6.59it/s]


...

Epoch 17: Train Loss=0.0480, Train Acc=0.9848, Val Loss=0.0263, Val Acc=0.9920


Epoch 18 Train: 100%|██████████| 3385/3385 [08:29<00:00,  6.64it/s]


Epoch 18: Train Loss=0.0469, Train Acc=0.9852, Val Loss=0.0327, Val Acc=0.9889


Epoch 19 Train: 100%|██████████| 3385/3385 [08:29<00:00,  6.64it/s]


Epoch 19: Train Loss=0.0443, Train Acc=0.9864, Val Loss=0.0634, Val Acc=0.9780
Early stopping at epoch 19


In [ ]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/lmmov5q7hwb2hck3b48rp


In [ ]:
import os
os.chdir("/workspace")

In [ ]:
torch.save(model.state_dict(), "se_resnet50.pth")

In [ ]:
from sklearn.metrics import classification_report

# 假设测试集路径为：
test_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/test', transform=val_transform)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="Testing"):
            x, y = x.to(device), y.to(device)
            with autocast():
                outputs = model(x)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = correct / total
    print(f"\n✅ Test Accuracy: {acc:.4f}")
    return all_preds, all_labels

    # 重新构建模型（要和训练时结构完全一致）
model = build_se_resnet50(num_classes=38).to(device)
model.load_state_dict(torch.load("se_resnet50.pth", map_location=device))

# 调用测试
test_preds, test_labels = test(model, test_loader)



print(classification_report(test_labels, test_preds, digits=4))


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Testing:   0%|          | 0/172 [00:00<?, ?it/s]/tmp/ipykernel_1277/2479971464.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Testing: 100%|██████████| 172/172 [00:10<00:00, 16.40it/s]


✅ Test Accuracy: 0.9802
              precision    recall  f1-score   support

           0     1.0000    0.9545    0.9767        66
           1     1.0000    1.0000    1.0000        63
           2     1.0000    1.0000    1.0000        32
           3     0.9940    1.0000    0.9970       165
           4     1.0000    1.0000    1.0000       151
           5     1.0000    1.0000    1.0000       106
           6     1.0000    0.9885    0.9942        87
           7     0.9091    0.9615    0.9346        52
           8     1.0000    0.9919    0.9959       123
           9     0.9794    0.9596    0.9694        99
          10     1.0000    1.0000    1.0000       117
          11     0.9916    1.0000    0.9958       118
          12     1.0000    0.9928    0.9964       139
          13     1.0000    1.0000    1.0000       108
          14     1.0000    1.0000    1.0000        43
          15     1.0000    0.9946    0.9973       552
          16     0.9868    0.9697    0.9782       231
  

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
import swanlab
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

swanlab.init(project="plant-disease-detection", run="se_resnet50_new3")

# 定义SE模块
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# 包装 torchvision 自带 Bottleneck 加 SE
class SEBottleneck(nn.Module):
    def __init__(self, bottleneck):
        super(SEBottleneck, self).__init__()
        self.bottleneck = bottleneck
        self.se = SEBlock(bottleneck.conv3.out_channels)

    def forward(self, x):
        out = self.bottleneck(x)
        out = self.se(out)
        return out

# 构建 SE-ResNet50
def build_se_resnet50(num_classes):
    model = models.resnet50(pretrained=False)
    for layer_name in ['layer1', 'layer2', 'layer3', 'layer4']:
        old_layer = getattr(model, layer_name)
        new_blocks = []
        for block in old_layer:
            new_blocks.append(SEBottleneck(block))
        setattr(model, layer_name, nn.Sequential(*new_blocks))
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Albumentations 增强定义
train_transform = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])


# 自定义 Dataset 兼容 Albumentations
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        from torchvision.datasets import ImageFolder
        self.dataset = ImageFolder(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

train_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/train', transform=train_transform)
val_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/val', transform=val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=36,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_se_resnet50(num_classes=38).to(device)
model.load_state_dict(torch.load("/workspace/se_resnet50.pth"))


# 2. 损失函数加权
from sklearn.utils.class_weight import compute_class_weight
labels = [label for _, label in train_dataset.dataset.samples]
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

scaler = GradScaler()

def train(model, train_loader, val_loader, epochs=20):
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss_total, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    out = model(x)
                    loss = criterion(out, y)
                val_loss_total += loss.item()
                val_correct += (out.argmax(1) == y).sum().item()
                val_total += y.size(0)
        val_loss = val_loss_total / len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step()

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
        swanlab.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })
        
         # 添加以下逻辑保存最佳模型
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "/workspace/best_se_resnet50.pth")

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

train(model, train_loader, val_loader, epochs=20)
torch.save(model.state_dict(), "se_resnet50_2.pth")


swanlab: Tracking run with swanlab version 0.6.0                                   
swanlab: Run data will be saved locally in /workspace/swanlog/run-20250604_110843-44500622
swanlab: 👋 Hi SZY_230507, welcome to swanlab!
swanlab: Syncing run horse-8 to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/px7ovcu02nnslqkqe285l


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1277/162202573.py:151: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1 Train:   0%|          | 0/3385 [00:00<?, ?it/s]/tmp/ipykernel_1277/162202573.py:161: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 Train: 100%|██████████|

Epoch 1: Train Loss=0.0421, Train Acc=0.9865, Val Loss=0.0271, Val Acc=0.9913


Epoch 2 Train: 100%|██████████| 3385/3385 [08:27<00:00,  6.67it/s]


Epoch 2: Train Loss=0.0364, Train Acc=0.9887, Val Loss=0.0403, Val Acc=0.9871


...

Epoch 16: Train Loss=0.0024, Train Acc=0.9993, Val Loss=0.0011, Val Acc=0.9997


Epoch 17 Train: 100%|██████████| 3385/3385 [08:26<00:00,  6.69it/s]


Epoch 17: Train Loss=0.0018, Train Acc=0.9995, Val Loss=0.0014, Val Acc=0.9998


Epoch 18 Train: 100%|██████████| 3385/3385 [08:25<00:00,  6.69it/s]


Epoch 18: Train Loss=0.0016, Train Acc=0.9996, Val Loss=0.0013, Val Acc=0.9997
Early stopping at epoch 18


In [ ]:
torch.save(model.state_dict(), "/workspace/se_resnet50_new.pth")

In [ ]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@SZY_230507/plant-disease-detection
swanlab: 🚀 View run at https://swanlab.cn/@SZY_230507/plant-disease-detection/runs/px7ovcu02nnslqkqe285l


In [ ]:
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
import swanlab
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# 定义SE模块
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        w = self.pool(x).view(b, c)
        w = self.fc(w).view(b, c, 1, 1)
        return x * w

# 包装 torchvision 自带 Bottleneck 加 SE
class SEBottleneck(nn.Module):
    def __init__(self, bottleneck):
        super(SEBottleneck, self).__init__()
        self.bottleneck = bottleneck
        self.se = SEBlock(bottleneck.conv3.out_channels)

    def forward(self, x):
        out = self.bottleneck(x)
        out = self.se(out)
        return out

# 构建 SE-ResNet50
def build_se_resnet50(num_classes):
    model = models.resnet50(pretrained=False)
    for layer_name in ['layer1', 'layer2', 'layer3', 'layer4']:
        old_layer = getattr(model, layer_name)
        new_blocks = []
        for block in old_layer:
            new_blocks.append(SEBottleneck(block))
        setattr(model, layer_name, nn.Sequential(*new_blocks))
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        from torchvision.datasets import ImageFolder
        self.dataset = ImageFolder(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_dataset = AlbumentationsDataset('/workspace/combined_dataset_split2/test', transform=val_transform)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=36,
    pin_memory=True
)
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="Testing"):
            x, y = x.to(device), y.to(device)
            with autocast():
                outputs = model(x)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = correct / total
    print(f"\n✅ Test Accuracy: {acc:.4f}")
    return all_preds, all_labels

    # 重新构建模型（要和训练时结构完全一致）
model = build_se_resnet50(num_classes=38).to(device)
model.load_state_dict(torch.load("/workspace/se_resnet50_new.pth", map_location=device))

# 调用测试
test_preds, test_labels = test(model, test_loader)



print(classification_report(test_labels, test_preds, digits=4))


Testing:   0%|          | 0/172 [00:00<?, ?it/s]

/tmp/ipykernel_109076/1074970552.py:101: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Testing: 100%|██████████| 172/172 [00:10<00:00, 15.72it/s]


✅ Test Accuracy: 0.9995
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        66
           1     1.0000    1.0000    1.0000        63
           2     1.0000    1.0000    1.0000        32
           3     1.0000    1.0000    1.0000       165
           4     1.0000    1.0000    1.0000       151
           5     1.0000    1.0000    1.0000       106
           6     1.0000    1.0000    1.0000        87
           7     1.0000    0.9808    0.9903        52
           8     1.0000    1.0000    1.0000       123
           9     0.9900    1.0000    0.9950        99
          10     1.0000    1.0000    1.0000       117
          11     1.0000    1.0000    1.0000       118
          12     1.0000    1.0000    1.0000       139
          13     1.0000    1.0000    1.0000       108
          14     1.0000    1.0000    1.0000        43
          15     1.0000    1.0000    1.0000       552
          16     1.0000    1.0000    1.0000       231
  